In [16]:
import pandas as pd

Importo un file json contenente le parole che mi interessa sezionare



In [17]:
words = pd.read_json('italian_words.json')

Creo due colonne; nella prima metto le prime due lettere della parola, nella seconda le prime tre

In [18]:
words['terzine'] = words[0].str[:3]

In [19]:
words['coppie'] = words[0].str[:2]

In [20]:
words.rename(columns={0:'word'}, inplace=True)

In [21]:
words

,word,terzine,coppie
0,abaca,aba,ab
1,abachi,aba,ab
2,abachista,aba,ab
3,abachiste,aba,ab
4,abachisti,aba,ab
...,...,...,...
661558,zwinglismo,zwi,zw
661559,zwinglista,zwi,zw
661560,zwingliste,zwi,zw
661561,zwinglisti,zwi,zw


cambio l'indice del dataframe

In [22]:
words.set_index('word')

,terzine,coppie
word,,
abaca,aba,ab
abachi,aba,ab
abachista,aba,ab
abachiste,aba,ab
abachisti,aba,ab
...,...,...
zwinglismo,zwi,zw
zwinglista,zwi,zw
zwingliste,zwi,zw


Metto in una serie pandas tutte le terzine e le coppie generate poi tolgo i duplicati

In [23]:
combinations = words.coppie.append(words.terzine)

In [24]:
combinations.drop_duplicates(keep='first', inplace=True)

In [25]:
combinations

0          ab
4586       ac
12773      ad
16328      ae
16722      af
         ... 
661446    zup
661493    zur
661539    zuz
661551    zwi
661562    zzz
Length: 2948, dtype: object

In [26]:
combinations.size

2948

In [27]:
combinations = pd.DataFrame(combinations, columns= ['letter_set'])

L'obiettivo è scartare le coppie e terzine che sono poco utilizzate nell'alfabeto. per ogni terzina/coppia conto in quante parole accade in modo da poterne scartare una parte

In [28]:
def search_letterset_in_words(x, columns_series):
  return columns_series.str.contains(x, regex = False).sum()

In [29]:
combinations['frequency'] = combinations['letter_set'].apply(search_letterset_in_words, args = (words['word'], ))

In [30]:
combinations[combinations['frequency'] > 500]

,letter_set,frequency
0,ab,14510
4586,ac,35403
12773,ad,12659
16328,ae,1565
16722,af,11731
...,...,...
659845,zin,1521
660117,zio,8573
660204,zir,573
660502,zol,1738


In [31]:
combinations[combinations['frequency'] > 500]['letter_set'].to_json('combinations.json', orient='records', lines=True)

In [32]:
combinations_normal = combinations[combinations['frequency'] > 500]

In [37]:
from sklearn import preprocessing

x = combinations_normal['frequency'].values #returns a numpy array

In [57]:
scaler = preprocessing.StandardScaler().fit(combinations_normal[['frequency']])
combinations_normal['frequency_std_scaled'] = scaler.transform(combinations_normal[['frequency']])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [58]:

min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(combinations_normal[['frequency_std_scaled']])
combinations_normal['difficulty'] =  x_scaled

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [59]:
combinations_normal['difficulty'] = 1 - combinations_normal['difficulty']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [60]:
combinations_normal[['letter_set','difficulty']]

,letter_set,difficulty
0,ab,0.922350
4586,ac,0.806526
12773,ad,0.932611
16328,ae,0.994113
16722,af,0.937756
...,...,...
659845,zin,0.994357
660117,zio,0.955263
660204,zir,0.999612
660502,zol,0.993154


In [64]:
combinations_normal['quantile'] = pd.qcut(combinations_normal['difficulty'], 4, labels=False)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [66]:
combinations_normal[['letter_set','quantile']]

,letter_set,quantile
0,ab,0
4586,ac,0
12773,ad,0
16328,ae,2
16722,af,0
...,...,...
659845,zin,2
660117,zio,0
660204,zir,3
660502,zol,2


In [69]:
combinations_normal[['letter_set','quantile']].to_json('combinations_and_levels.json', orient='records', lines=True)

# Nuova sezione